In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define paths to your train and test datasets
train_data_dir = '/kaggle/input/geometric-shapes-circle-square-triangle/train'
test_data_dir = '/kaggle/input/geometric-shapes-circle-square-triangle/test'



# Image size and channels
img_width, img_height, img_channels = 56, 56, 3

In [ ]:
# Createing a data generator with data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Rescale pixel values to be between 0 and 1
    rotation_range=30,           # Randomly rotate images in the range of -30 to +30 degrees
    width_shift_range=0.1,       # Randomly shift images horizontally by 10%
    height_shift_range=0.1,      # Randomly shift images vertically by 10%
    zoom_range=0.2,              # Randomly zoom into images by 20%
    horizontal_flip=True        # Randomly flip images horizontally
)

# Only rescaling for the test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Generateing batches of augmented data for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'    # Use categorical mode for multi-class classification
)

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'    # Use categorical mode for multi-class classification
)

In [ ]:
# Building the CNN Model
model = models.Sequential([
    # Convolutional layer with 32 filters, each of size (3, 3), and ReLU activation function
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, img_channels)),
    
    # MaxPooling layer to down-sample the spatial dimensions
    layers.MaxPooling2D((2, 2)),
    
    # Convolutional layer with 64 filters, each of size (3, 3), and ReLU activation function
    layers.Conv2D(64, (3, 3), activation='relu'),
    
    # MaxPooling layer to down-sample the spatial dimensions
    layers.MaxPooling2D((2, 2)),
    
    # Convolutional layer with 128 filters, each of size (3, 3), and ReLU activation function
    layers.Conv2D(128, (3, 3), activation='relu'),
    
    # Flatten layer to flatten the output of the previous layer
    layers.Flatten(),
    
    # Fully connected layer with 128 neurons and ReLU activation function
    layers.Dense(128, activation='relu'),
    
    # Output layer with 3 neurons (assuming 3 classes) and softmax activation function for multi-class classification
    layers.Dense(3, activation='softmax')
])

# Display the model summary
model.summary()

In [ ]:
# Compileing the model
model.compile(optimizer='adam',            # Optimizer: Adam
              loss='categorical_crossentropy',  # Loss function: Categorical Crossentropy
              metrics=['accuracy'])            # Evaluation metric: Accuracy

# Training the model
history = model.fit(train_generator,                  # Training data generator
          epochs=15,                        # Number of training epochs
          validation_data=validation_generator)  # Validation data generator

# Check the accuracy of the model on the test set
score = model.evaluate(validation_generator)  # Evaluate the model on the validation set
print(score)
print('Test loss:', score[0])                 # Display the test loss
print('Test accuracy:', score[1])             # Display the test accuracy


In [ ]:
# Import necessary library
import matplotlib.pyplot as plt

# Plotting the training and validation accuracy over epochs
plt.plot(history.history['accuracy'], color='red', label='train')         # Plotting training accuracy in red
plt.plot(history.history['val_accuracy'], color='blue', label='validation')  # Plotting validation accuracy in blue

plt.legend()
plt.show()

#The resulting plot will provide a visual representation of how well our model is learning from the training data and generalizing to unseen validation data over the training epochs.

In [ ]:
# Plotting the training and validation loss over epochs
plt.plot(history.history['loss'], color='red', label='train')          # Plot training loss in red
plt.plot(history.history['val_loss'], color='blue', label='validation')  # Plot validation loss in blue

plt.legend()
plt.show()

# The resulting plot provides a visual representation of how well the model is minimizing the loss function during training and validation.

In [ ]:
import cv2

In [ ]:
test_img1 = cv2.imread('/kaggle/input/geometric-shapes-circle-square-triangle/Real Images/triangle.png')
test_img2 = cv2.imread('/kaggle/input/geometric-shapes-circle-square-triangle/Real Images/circle.png')
test_img3 = cv2.imread('/kaggle/input/geometric-shapes-circle-square-triangle/Real Images/square.png')


In [ ]:


plt.subplot(131)
plt.imshow(test_img1)
plt.title('test_img1')

plt.subplot(132)
plt.imshow(test_img2)
plt.title('test_img2')

plt.subplot(133)
plt.imshow(test_img3)
plt.title('test_img3')
plt.show()


In [ ]:
print(test_img1.shape)
print(test_img2.shape)
print(test_img3.shape)

In [ ]:
test_img1 = cv2.resize(test_img1,(56,56))
test_img2 = cv2.resize(test_img2,(56,56))
test_img3 = cv2.resize(test_img3,(56,56))


In [ ]:
# Converting to shape 56*56*3
test_image1 = test_img1.reshape((1,56,56,3))
test_image2 = test_img2.reshape((1,56,56,3))
test_image3 = test_img3.reshape((1,56,56,3))

In [ ]:
input_tensor = tf.concat([test_image1, test_image2, test_image3], axis=0)
predictions = model.predict(input_tensor)

In [ ]:
print(predictions)